# Análise das Dependências do Python

In [132]:
import pandas as pd
import requests
import urllib.request
import string
import numpy as np
import json
import collections
from bs4 import BeautifulSoup
import tqdm
import pickle
import time

%matplotlib inline
pd.set_option('display.max_rows', 200)

## 1. Web-Scraping dos Dados

### 1.1. Scraping da Relação de Pacotes Científicos

Para o trabalho serão utilizados os pacotes científicos distribuidos pelo Anaconda. Na documentação do Ananconda são relacionados os pacotes de acordo com a versão do Python: 2.7, 3.4, 3.5 e 3.6

In [133]:
#Carrega as tabela a partir do html

pkgs_py27 = pd.read_html('https://docs.continuum.io/anaconda/pkg-docs',skiprows=1)[0]
pkgs_py34 = pd.read_html('https://docs.continuum.io/anaconda/pkg-docs',skiprows=1)[1]
pkgs_py35 = pd.read_html('https://docs.continuum.io/anaconda/pkg-docs',skiprows=1)[2]
pkgs_py36 = pd.read_html('https://docs.continuum.io/anaconda/pkg-docs',skiprows=1)[3]

In [134]:
#retirar as informações desnecessarias: 
#sistema operacional do pacote
def limpa_campo_pkg(pkgs):
    
    pkgs.columns = ['pkg', 'version', 'summary', 'in_installer']
    
    pkgs['pkg'] = pkgs.pkg.apply(lambda x: str(x).replace('Mac',''))
    pkgs['pkg'] = pkgs.pkg.apply(lambda x: str(x).replace('Linux',''))
    pkgs['pkg'] = pkgs.pkg.apply(lambda x: str(x).replace('Windows',''))
    pkgs['pkg'] = pkgs.pkg.apply(lambda x: str(x).rstrip())
    
    return pkgs

In [135]:
pkgs_py27 = limpa_campo_pkg(pkgs_py27)
pkgs_py34 = limpa_campo_pkg(pkgs_py34)
pkgs_py35 = limpa_campo_pkg(pkgs_py35)
pkgs_py36 = limpa_campo_pkg(pkgs_py36)

Para a obtenção dos links referentes a cada pacote que estão na tabela de pacotes, o método anterior não é efetivo. Será utilizado o Beautiful Soup 4.

In [136]:
def make_soup(url):
    the_page = urllib.request.urlopen(url)
    soup = BeautifulSoup(the_page,"html.parser")
    
    return soup

In [137]:
soup = make_soup('https://docs.continuum.io/anaconda/pkg-docs')

In [138]:
soup

<!DOCTYPE html>

<html class="no-js" dir="ltr" lang="en">
<head>
<meta content="width=device-width, initial-scale=1" name="viewport">
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible">
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><script type="text/javascript">window.NREUM||(NREUM={}),__nr_require=function(e,n,t){function r(t){if(!n[t]){var o=n[t]={exports:{}};e[t][0].call(o.exports,function(n){var o=e[t][1][n];return r(o||n)},o,o.exports)}return n[t].exports}if("function"==typeof __nr_require)return __nr_require;for(var o=0;o<t.length;o++)r(t[o]);return r}({1:[function(e,n,t){function r(){}function o(e,n,t){return function(){return i(e,[c.now()].concat(u(arguments)),n?null:this,t),n?void 0:this}}var i=e("handle"),a=e(2),u=e(3),f=e("ee").get("tracer"),c=e("loader"),s=NREUM;"undefined"==typeof window.newrelic&&(newrelic=s);var p=["setPageViewName","setCustomAttribute","setErrorHandler","finished","addToTrace","inlineHit","addRelease"],d="api-",l=d+"ixn

In [139]:
#obtem os links de cada pacote
links = []

for table in soup.find_all("table"):
    for registro in table.find_all("tr"):
        for link in registro.contents[1].find_all("a", href = True):
            links.append(link['href'])


Associando os links ao pacote correspondente

In [140]:
pkgs_py27['link'] = links[0:521]
pkgs_py34['link'] = links[521:(521+476)]
pkgs_py35['link'] = links[(521+476):(521+476+474)]
pkgs_py36['link'] = links[(521+476+474):(521+476+474+453)]

In [141]:
def arruma_df(pkg):

    pkg.columns = ['pkg', 'version', 'summary', 'in_installer', 'link']

    pkg = pkg.drop('in_installer', axis = 1)

    return pkg

In [142]:
pkgs_py27 = arruma_df(pkgs_py27)
pkgs_py34 = arruma_df(pkgs_py34)
pkgs_py35 = arruma_df(pkgs_py35)
pkgs_py36 = arruma_df(pkgs_py36)

Sample do resultado

In [143]:
pkgs_py36.sample(10)

,pkg,version,summary,link
61,colorcet,0.9.1,collection of perceptually uniform colormaps /...,http://github.com/bokeh/colorcet
217,more-itertools,2.2,"more routines for operating on iterables, beyo...",https://github.com/erikrose/more-itertools
5,anaconda-navigator,1.5.0,Anaconda Navigator / proprietary - Continuum A...,https://github.com/ContinuumIO/navigator
270,petl,1.1.0,/ MIT,https://github.com/alimanfoo/petl
48,chameleon,2.24,HTML/XML template engine for Python / BSD-like,https://chameleon.readthedocs.org
354,ruamel_yaml,0.11.14,patched copy of ruamel.yaml (a yaml parser) / MIT,https://bitbucket.org/ruamel/yaml
42,cachecontrol,0.11.7,httplib2 caching algorithms for use with reque...,https://github.com/ionrock/cachecontrol
317,pysal,1.12.0,library of spatial analysis functions / 3-clau...,http://pysal.org
13,apr,1.5.2,Maintains a consistent API with predictable be...,http://apr.apache.org/
332,pywin32,220,Python extensions for Windows / PSF,http://sourceforge.net/projects/pywin32/


Verificação se alguns pacotes principais tem o link do github referênciado

In [144]:

print(pkgs_py36[pkgs_py36['pkg'] == 'pandas'])
print(pkgs_py36[pkgs_py36['pkg'] == 'numpy'])
print(pkgs_py36[pkgs_py36['pkg'] == 'scikit-learn'])


        pkg version                                            summary  \
252  pandas  0.19.2  Powerful data structures and data analysis too...   

                          link  
252  http://pandas.pydata.org/  
       pkg version                                            summary  \
243  numpy  1.11.3  array processing for numbers, strings, records...   

                        link  
243  http://numpy.scipy.org/  
              pkg version                                            summary  \
361  scikit-learn  0.18.1  set of python modules for machine learning and...   

                                link  
361  http://scikit-learn.org/stable/  


Aparentemente pacotes importantes não estão como github associado. Trocaremos os links manualmente.

In [145]:
def corrige_link(pkg, url):
    
    pkgs_py36['link'][pkgs_py36['pkg'] == pkg] = url


In [207]:
corrige_link('pandas', 'https://github.com/pandas-dev/pandas')
corrige_link('numpy', 'https://github.com/numpy/numpy')
corrige_link('scikit-learn', 'https://github.com/scikit-learn/scikit-learn')
corrige_link('scikit-image', 'https://github.com/scikit-image/scikit-image')
corrige_link('scipy', 'https://github.com/scipy/scipy')
corrige_link('sympy', 'https://github.com/sympy/sympy')
corrige_link('plotly', 'https://github.com/plotly/plotly.py')
corrige_link('chest', 'https://github.com/blaze/chest')
corrige_link('clyent', 'https://github.com/Anaconda-Platform/clyent')
corrige_link('configargparse', 'https://github.com/bw2/ConfigArgParse')
corrige_link('cvxcanon', 'https://github.com/cvxgrp/CVXcanon')
corrige_link('cymem', 'https://github.com/explosion/cymem')
corrige_link('datrie', 'https://github.com/pytries/datrie')
corrige_link('incremental', 'https://github.com/twisted/incremental')
corrige_link('ipywidgets', 'https://github.com/jupyter-widgets/ipywidgets')
corrige_link('pickleshare', 'https://github.com/pickleshare/pickleshare')
corrige_link('preshed', 'https://github.com/explosion/preshed')
corrige_link('smart_open', 'https://github.com/RaRe-Technologies/smart_open')
corrige_link('zict', 'https://github.com/dask/zict')
corrige_link('chalmers', 'https://github.com/Anaconda-Platform/chalmers')



3 pacotes deram problema no link obtido devido ao comprimento da string. No primeiro momento esses links serão removidos.

In [211]:
#retira links quebras
corrige_link('anaconda-navigator', '')
corrige_link('sputnik', '')
corrige_link('semantic_version', '') #https://github.com/rbarrois/python-semanticversion

Resultado da coleta dos meta dados dos pacotes

In [149]:
print('py{} : {} / {} pacotes com endereço do git'.format( 
    "27",str(len(pkgs_py27[pkgs_py27['link'].str.contains("//github")])),str(len(pkgs_py27))))
print('py{} : {} / {} pacotes com endereço do git'.format( 
    "34",str(len(pkgs_py34[pkgs_py34['link'].str.contains("//github")])),str(len(pkgs_py34))))
print('py{} : {} / {} pacotes com endereço do git'.format( 
    "35",str(len(pkgs_py35[pkgs_py35['link'].str.contains("//github")])),str(len(pkgs_py35))))
print('py{} : {} / {} pacotes com endereço do git'.format( 
    "36",str(len(pkgs_py36[pkgs_py36['link'].str.contains("//github")])),str(len(pkgs_py36))))
      

py27 : 183 / 521 pacotes com endereço do git
py34 : 167 / 476 pacotes com endereço do git
py35 : 165 / 474 pacotes com endereço do git
py36 : 162 / 453 pacotes com endereço do git


Para todas as versões do Python, cerca de 1/3 tem o seu GitHub referênciado na documentação do Anaconda. Vamos crawlear esses pacotes.

### 1.2. Scraping das Dependências dos Pacotes

Definição das funções para scraping

In [247]:
#Coleção de Funções que serão utilizadas no git_code_crawler

## Função para acessar a API
def git_code_qry(repository, keyword, language = 'python', results_per_page = 100, page = 1):
    headers = {
        'Accept': 'application/vnd.github.v3.text-match+json',
    }

    params = (
        ('q', keyword + ' language:' + language + ' repo:' + repository),
        ('per_page', str(results_per_page)),
        ('page', str(page)),
    )

    return requests.get('https://api.github.com/search/code', headers=headers, params=params)

## A API do GitHub retorna 100 resultados por consulta. Para casos onde há mais resultados será necessário uma
## que verifica a quantidade de páginas com resultado. Essa informação será parâmetro da consulta em eventuais 
## casos onde há mais de 100 resultados
##Função para obtenção da quantidade de páginas retornadas na consulta
def page_qtt(query):
    try:
        last_page = int(query.headers['Link'].split(';')[1][-2])
    except:    
        last_page = 1
    
    return last_page


##Função para obter a setença com onde foi encotrada a palavra chave
def get_text(query):
    
    imports = []
    
    for item in json.loads(query.text)['items']:
        for text in (item['text_matches']):
            imports.append(text['fragment'])
    
    return imports


##Função para limpar o texto que retornou da consulta
def remove_regex(qry_text):
    #remover regex
    qry_text_noregex = []
    
    for sentence in qry_text:
        sentence_noregex = sentence
        sentence_noregex = sentence_noregex.replace("  ", " ")
        sentence_noregex = sentence_noregex.replace("\n", " ")
        sentence_noregex = sentence_noregex.replace("\r", " ")
        qry_text_noregex.append(sentence_noregex)
    
    return qry_text_noregex
    

In [168]:
##Função que acessa a API e retorna o json

def git_code_crawler(repository, keyword, language = 'python', results_per_page = 100):
    
    results = []
    
    #first query
    qry = git_code_qry(repository, keyword, language, results_per_page, 1)
    
    results.extend(get_text(qry))
    results_qtt = len(json.loads(qry.text)['items'])
    
    #Verify if query return more than one page
    if page_qtt(qry) != 1:
        
        for i in range(2, page_qtt(qry) + 1):
            
            qry = git_code_qry(repository, keyword, language, results_per_page, i)
            
            results.extend(get_text(qry))
            results_qtt += len(json.loads(qry.text)['items'])

    #print('{} : {} resultados encontrados!'.format(repository, results_qtt))
    
    return results

In [156]:
#Função para obter do json a informação dos pacotes acessados pelo repositório
def get_pkgs(qry_text, most_common = 1000):
    
    #remover regex
    qry_text_noregex = []
    for sentence in qry_text:
        qry_text_noregex.append(sentence.replace("  ", " ").replace("\n", " "))
    
    #splita as lista
    qry_split = []

    for sentence in qry_text_noregex:
        qry_split.append(sentence.split(" "))
        
    #remove valores vazios da lista
    qry_no_blank_space = []
    
    chars = set('"#`_-()')
    
    for sentence in qry_split:
        for word in sentence:
            if word != "" and word != "#" and word != "," and word != '"""' and word != '"':
                if not any((c in chars) for c in word):
                    word = word.replace(",", "")
                    qry_no_blank_space.append(word)
    
    #obtem os pacotes
    pacotes = []

    for i in range(0,len(qry_no_blank_space)-1):
        
        if qry_no_blank_space[i] == 'from':
            pacotes.append(qry_no_blank_space[i+1])
        
        elif qry_no_blank_space[i] == 'import' and qry_no_blank_space[i - 2] != 'from':
            pacotes.append(qry_no_blank_space[i+1])
            
        elif qry_no_blank_space[i] == 'cimport' and qry_no_blank_space[i - 2] != 'from':
            pacotes.append(qry_no_blank_space[i+1])
    
    #Limpa a os pacotes filhos
    pacotes_limpos = [] 
    
    for pkg in pacotes:
        if pkg.find('.') > 0:
            pacotes_limpos.append(pkg[0:pkg.find('.')])
        elif pkg[0] != '.' and pkg[0] != '-' and pkg[0] != '`' and pkg[0] != '('  and pkg[0] != '#'  and pkg[0] != '"' and pkg[0] != '_' and pkg[0:2] != 'is' and pkg[0].islower() and pkg != 'import' and pkg != 'from' and pkg != 'the' and pkg != 'division':
            pacotes_limpos.append(pkg)
    
    return collections.Counter(pacotes_limpos).most_common(most_common)
    

Funções do crawler todas definidas.

Agora será obtido o nome do repositório a partir da url do github. Essa informação é um dos parâmetros para consulta na API.

In [229]:
repositorios = []

for repositorio in pkgs_py36[pkgs_py36['link'].str.contains("//github")]['link']:
    
    if repositorio.find('https')>-1:
        pacote = repositorio.replace("https://github.com/", "")
    
        if pacote[-1] == '/':
            pacote = pacote[:-1]
            repositorios.append(pacote)
        else:
            repositorios.append(pacote) 
              
    else:
        pacote = repositorio.replace("http://github.com/", "")
        
        if pacote[-1] == '/':
            pacote = pacote[:-1]
            repositorios.append(pacote)
        else:
            repositorios.append(pacote) 
        

Execução do crawler.

Os parâmetros utilizados são:

1) Nome do repositório buscado

2) Palavra 'import' no código

3) Estar escrito em py

In [274]:
pacote_pacote = {}

In [277]:

for repo in tqdm.tqdm(repositorios):
        #print(repo)
        pkg_name = repo[repo.find('/')+1 : ]
        result = git_code_crawler(repo, "import", language = 'python')
        dependencies = get_pkgs(result)
        
        pacote_pacote[pkg_name] = dependencies
        
        time.sleep(15)

100%|██████████| 1/1 [00:21<00:00, 21.56s/it]


Salva o pickle da consulta

In [1075]:
pickle.dump(pacote_pacote,open("dependencias.p", 'wb'))


In [280]:
pacote_pacote = pickle.load(open('dependencias.p', 'rb'))

Visualização do resultado do Crawler.

Os valores são a quantidade de menções ao pacote dentro do repositório

In [281]:
pacote_pacote

{'CVXcanon': [('numpy', 34),
  ('cvxpy', 30),
  ('time', 21),
  ('matplotlib', 17),
  ('copy', 6),
  ('math', 5),
  ('os', 3),
  ('mpl_toolkits', 3),
  ('scipy', 3),
  ('glob', 1),
  ('subprocess', 1),
  ('heapq', 1),
  ('setuptools', 1),
  ('huge_testman', 1),
  ('unittest', 1),
  ('again', 1),
  ('collections', 1),
  ('sys', 1),
  ('imp', 1)],
 'ConfigArgParse': [('sys', 3),
  ('logging', 2),
  ('os', 2),
  ('argparse', 2),
  ('types', 2),
  ('setuptools', 1),
  ('setup', 1),
  ('configargparse', 1),
  ('functools', 1),
  ('inspect', 1),
  ('tempfile', 1),
  ('unittest2', 1),
  ('glob', 1),
  ('re', 1),
  ('io', 1)],
 'Fiona': [('fiona', 102),
  ('logging', 37),
  ('sys', 34),
  ('os', 25),
  ('click', 22),
  ('json', 20),
  ('pytest', 19),
  ('unittest', 18),
  ('tempfile', 15),
  ('shutil', 14),
  ('cligj', 9),
  ('six', 6),
  ('collections', 5),
  ('subprocess', 5),
  ('warnings', 4),
  ('re', 4),
  ('functools', 4),
  ('argparse', 3),
  ('timeit', 3),
  ('osgeo', 3),
  ('collecti

### 1.3. Formata o arquivo csv que será carregado no Gephi 

Gera as matriz utilizada no grapho

Obtém toda a relação de pacotes: dependentes (pais) e depedencias (filhos).

Foi utilizado esse nome devido a a releção hierárquica dentro do dicionário. Essa nomeclatura poderia ser melhorada.

In [288]:
#obtém o conjunto de todos os pacotes-pai (dependetes)
pacotes_pai = set(pacote_pacote.keys())
print(pacotes_pai)


{'clyent', 'colorama', 'patsy', 'multipledispatch', 'html5lib-python', 'feedparser', 'pyopenssl', 'nb_conda_kernels', 'pandas-datareader', 'conda-verify', 'partd', 'conda/wiki/VC-features', 'cytoolz', 'luigi', 'ConfigArgParse', 'qtpy', 'appnope', 'apptools', 'pyserial', 'scipy', 'azure-sdk-for-python', 'traitsui', 'wrapt', 'queuelib', 'rasterio', 'keyring', 'petl', 'PyTd', 'flask-login', 'bleach', 'libconda', 'PyMySQL', 'pystan', 'CVXcanon', 'traits', 'path.py', 'python-unicodecsv', 'responses', 'XlsxWriter', 'thinc', 'PyHive', 'constantly', 'colander', 'kerberos-sspi', 'smart_open', 'anaconda-clean', 'ipywidgets', 'redis-py', 'nb_anacondacloud', 'stripe-python', 'win-unicode-console', 'greenlet', 'runipy', 'cachecontrol', 'backports.shutil_get_terminal_size', 'wcwidth', 'constructor', 'pyface', 'pytest-cov', 'numpy', 'pandas', 'cloudpickle', 'hs2client', 'trollius', 'traitlets', 'numexpr', 'qtawesome', 'more-itertools', 'numpydoc', 'Shapely', 'rope', 'scikit-image', 'requests-kerberos

In [289]:
#obtém o conjunto de pacotes filhos (dependencias)
pacotes_filhos = []

for pai in pacote_pacote:
    for filho in pacote_pacote[pai]:
        pacotes_filhos.append(filho[0])

pacotes_filhos = set(pacotes_filhos)

print(len(pacotes_filhos))
print(pacotes_filhos)

1236
{'abstract_type_system', 'sha', 'clyent', 'os', 'filtering_handler', 'time\r', 'trait_dict_context_adapter', 'read_config', 'use_solver', 'coding:', 'multipledispatch', 'object', 'urlparse', ':`grid_search`', 'read_configuration', 'do_later', 'match_template', 'intranges_from_list', 'partd', 'gzip', 'alloc_hook', 'UTS46', 'your', 'msrest', 'pkg1', 'pyhive', 'build', 'nodes', 'open_file', 'pyfits', 'escape_bytes', 'SGETask', 'evaluate_trait', 'scipy', 'http://json', 'basename', 'msgpack', 'traitsui', 'provides', 'win32security', 'binarization', ':mod:`scipy', 'json', 'prompt_toolkit', 'assert_raises', 'abstract_adapter_factory', 'weakref\r', 'gen_backend', 'manager', 'contrib', 'fields', 'convert', 'build_control', 'anaconda_project', 'uuid', 'readline', 'iso8601', 'json\r', 'path_for', 'kernprof', 'difflib', 'collections\r', 'load_breast_cancer', 'minpack', 'login_action', 'winreg', 'cryptography', 'exception', 'make_blobs', 'csr_matrix', 'for', 'pdb', 'webencodings', 'local', 'he

Mergea pais e filhos para termos todos os nós

In [423]:
pacotes_total = pacotes_pai.union(pacotes_filhos)

In [304]:
pacotes_total2 = list(pacotes_total)
pacotes_total2.sort()
pacotes_total2

['""":func:`~pandas',
 '"wtforms',
 "(r'",
 ')',
 '*"',
 '/kddcup99-mld/kddcup',
 '/licenses/>',
 '/licenses/BSD-3-Clause',
 '/mixture/dpgmm',
 '/test_layer',
 '://github',
 '://pyserial',
 '://www',
 ':`grid_search`',
 ':mod:`scipy',
 '<alexandre',
 '<arnaud',
 '<s8wu@uwaterloo',
 '@inria',
 '@normalesup',
 'AWS',
 'CVXcanon',
 'ConfigArgParse',
 'Crypto',
 'Cython',
 'Distribution',
 'Fiona',
 'GCS',
 'IPython',
 'Interface',
 'License',
 'Magic',
 'Numeric',
 'Oct',
 'OpenSSL',
 'PyHive',
 'PyMySQL',
 'PyQt4',
 'PyQt5',
 'PySide',
 'PySide2',
 'PyTd',
 'Pyro',
 'SGETask',
 'SOFTWARE',
 'SONManipulators',
 'Shapely',
 'System',
 'TableEditor',
 'UTS46',
 'V',
 'XlsxWriter',
 '__init__',
 '_root',
 '_version',
 '`',
 '``PyQt5',
 '`sys',
 '`tornado',
 'a',
 'a2b_hex',
 'abc',
 'absolute_import',
 'abspath',
 'abstract_adapter_factory',
 'abstract_factory',
 'abstract_type_system',
 'action',
 'actions',
 'adapt',
 'adaptable',
 'adapter_base',
 'adapter_manager',
 'add_parser_json',
 '

In [314]:
indice = list(range(0,len(pacotes_total2)))

In [422]:
len(indice)

1302

Formatação dos dados para o formato que o Gephi recebe.

O Gephi precisa de duas bases diferentes:

1) Relação dos nós (nodes)

2) Relação das arestas (edges)

In [427]:
#cria os nodes:
db_nodes = pd.DataFrame({'ID' : indice, 'Label' : pacotes_total2})
db_nodes

,ID,Label
0,0,""""""":func:`~pandas"
1,1,"""wtforms"
2,2,(r'
3,3,)
4,4,"*"""
5,5,/kddcup99-mld/kddcup
6,6,/licenses/>
7,7,/licenses/BSD-3-Clause
8,8,/mixture/dpgmm
9,9,/test_layer


Aqui observamos que ainda há algumas sujeiras. Elas vieram da relação de filhos (dependencias).

Como aparentam ser poucos, neste momento, serão tratados manualmente dentro do Gephi.

In [441]:
#cria os edges
db_edges = pd.DataFrame({'Label' : [], 'Label2' : []})
db_edges


,Label,Label2


In [442]:
#preenche o dataframe de edges

for pai in pacote_pacote:
    for filho in pacote_pacote[pai]:
        #verifica se o pacote se referência a ele mesmo
        if pai != filho[0]:
                db_edges = db_edges.append(pd.DataFrame([[pai, filho[0]]], columns=['Label', 'Label2']))




In [443]:
db_edges

,Label,Label2
0,affine,math
0,affine,pickle
0,affine,multiprocessing
0,affine,codecs
0,affine,setuptools
0,affine,unittest
0,affine,textwrap
0,affine,nose
0,affine,collections
0,anaconda-clean,sys


In [444]:
db_edges = pd.merge(db_edges, db_nodes, how='left', on = 'Label')
db_edges = db_edges[['ID', 'Label2']]
db_edges.columns = ['Source','Label']
db_edges = pd.merge(db_edges, db_nodes, how='left', on = 'Label')
db_edges = db_edges[['Source', 'ID']]
db_edges.columns = ['Source','Target']
db_edges['Type'] = "Direct"
db_edges['Weight'] = 1
db_edges

,Source,Target,Type,Weight
0,78,683,Direct,1
1,78,835,Direct,1
2,78,724,Direct,1
3,78,213,Direct,1
4,78,1023,Direct,1
5,78,1219,Direct,1
6,78,1149,Direct,1
7,78,753,Direct,1
8,78,221,Direct,1
9,85,1118,Direct,1


Salva os csv`s!

In [446]:
db_nodes.to_csv(path_or_buf='db_nodes.csv', sep=';',na_rep=0, index=False)
db_edges.to_csv(path_or_buf='db_edges.csv', sep=';',na_rep=0,  index=False)